In [ ]:
import numpy as np
import scipy as sp

import tensorflow as tf
from tensorflow import keras

import tf2onnx
import onnx
import os

## Choose computation device (CPU)

In [ ]:
physical_devices = tf.config.list_physical_devices()
print(f"These are the physical devices available:\n{physical_devices}")

try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    print(f"These are the visible devices:\n{visible_devices}")
except:
    pass

## User inputs

In [ ]:
# EDIT THIS SECTION FOR USER INPUTS
#
name = 'model_0'
in_file = '../data/ts9_test1_in_FP32.wav'
out_file = '../data/ts9_test1_out_FP32.wav'
epochs = 1

train_mode = 0     # 0 = speed training, 
                   # 1 = accuracy training 
                   # 2 = extended training

input_size = 150 
batch_size = 4096 
test_size = 0.2

if not os.path.exists('models/'+name):
    os.makedirs('models/'+name)
else:
    print("A model with the same name already exists. Please choose a new name.")
    exit

## Define some helper functions

In [ ]:
def save_wav(name, data):
    sp.io.wavfile.write(name, 44100, data.flatten().astype(np.float32))

def normalize(data):
    data_max = max(data)
    data_min = min(data)
    data_norm = max(data_max,abs(data_min))
    return data / data_norm

## Pre-processing the data

In [ ]:
# Load and Preprocess Data ###########################################
in_rate, in_data = sp.io.wavfile.read(in_file)
out_rate, out_data = sp.io.wavfile.read(out_file)

X_all = in_data.astype(np.float32).flatten()  
X_all = normalize(X_all).reshape(len(X_all),1)   
y_all = out_data.astype(np.float32).flatten() 
y_all = normalize(y_all).reshape(len(y_all),1)

# Get the last 20% of the wav data for testing and thee rest for training
X_training, X_testing = np.split(X_all, [int(len(X_all)*(1-test_size))])
y_training, y_testing = np.split(y_all, [int(len(y_all)*(1-test_size))])
print(f"X_training shape (pre-processing): {X_training.shape}")
print(f"y_training shape (pre-processing): {y_training.shape}")
print(f"X_testing shape (pre-processing): {X_testing.shape}")
print(f"y_testing shape (pre-processing): {y_testing.shape}")

# Create a new array where each element is an array of input_size samples in time order
# Each element of the new array is shifted by one sample from the previous element
indices = np.arange(input_size) + np.arange(len(X_training)-input_size+1)[:,np.newaxis]

# We need to split the data otherwise the tf.gather function will throw an error since we run out of memory
X_ordered_training = tf.gather(X_training,indices[:len(indices)//2])
X_ordered_training = tf.concat([X_ordered_training, tf.gather(X_training,indices[len(indices)//2:])], axis=0)
print(f"X_ordered_training shape: {X_ordered_training.shape}")

indices = np.arange(input_size) + np.arange(len(X_testing)-input_size+1)[:,np.newaxis]
X_ordered_testing = tf.gather(X_testing,indices) 
print(f"X_ordered_testing shape: {X_ordered_testing.shape}")

# The input size defines the number of samples used for each prediction
# Therefore the first output value that we get is at index input_size-1
y_ordered_training = y_training[input_size-1:]
print(f"y_ordered_training shape: {y_ordered_training.shape}")
y_ordered_testing = y_testing[input_size-1:]
print(f"y_ordered_testing shape: {y_ordered_testing.shape}")


shuffled_indices = np.random.permutation(len(X_ordered_training)) 
X_random_training = tf.gather(X_ordered_training, shuffled_indices)
y_random_training = tf.gather(y_ordered_training, shuffled_indices)
print(f"X_random_training shape (post-processing): {X_random_training.shape}")
print(f"y_random_training shape (post-processing): {y_random_training.shape}")

print(f"The X_random_training data is an array, where each element is an array of input_size samples in time order. Therefore the lenght is smaller than the original X_training array (the first {input_size} samples are grouped).")
print(f"The y_random_training data is an array, where each element is a single sample. This single sample is the target output for the corresponding X_random_training element, which consists of input samples.")

## Define the model

In [ ]:
'''This is a similar Tensorflow/Keras implementation of the LSTM model from the paper:
    "Real-Time Guitar Amplifier Emulation with Deep Learning"
    https://www.mdpi.com/2076-3417/10/3/766/htm

    Uses a stack of two 1-D Convolutional layers, followed by LSTM, followed by 
    a Dense (fully connected) layer. Three preset training modes are available, 
    with further customization by editing the code. A Functional keras model 
    is implemented here.

    Note: RAM may be a limiting factor for the parameter "input_size". The wav data
      is preprocessed and stored in RAM, which improves training speed but quickly runs out
      if using a large number for "input_size".  Reduce this if you are experiencing
      RAM issues.
'''

if train_mode == 0:         # Speed Training
    learning_rate = 0.01 
    conv1d_strides = 12   
    conv1d_1_strides = 12
    conv1d_filters = 16
    hidden_units = 36
elif train_mode == 1:       # Accuracy Training (~10x longer than Speed Training)
    learning_rate = 0.01 
    conv1d_strides = 4
    conv1d_filters = 36
    hidden_units= 64
else:                       # Extended Training (~60x longer than Accuracy Training)
    learning_rate = 0.0005 
    conv1d_strides = 3
    conv1d_filters = 36
    hidden_units= 96

# Create Functional Model ###########################################
keras.backend.clear_session()

class NeuralNetwork(tf.keras.Model):
    def __init__(self, input_size, conv1d_filters, conv1d_strides, hidden_units, learning_rate):
        super(NeuralNetwork, self).__init__()
        self.conv1d_1 = keras.layers.Conv1D(filters=conv1d_filters, kernel_size=12, strides=conv1d_strides, activation=None, batch_size=1)
        self.conv1d_2 = keras.layers.Conv1D(filters=conv1d_filters, kernel_size=12, strides=conv1d_strides, activation=None, batch_size=1)
        self.lstm = keras.layers.LSTM(units=hidden_units, return_sequences=False, stateful=False, batch_size=batch_size, kernel_initializer='glorot_uniform', recurrent_initializer='zeros', bias_initializer='zeros', use_bias=True)
        self.dense = keras.layers.Dense(units=1, activation=None, batch_size=1)

        self.build((1, input_size, 1))

    def call(self, inputs, training=False):
        x = keras.layers.ZeroPadding1D(padding=12, batch_size=1)(inputs)
        x = self.conv1d_1(x)
        x = keras.layers.ZeroPadding1D(padding=12, batch_size=1)(x)
        x = self.conv1d_2(x)
        x = self.lstm(x)
        return self.dense(x)

# Define the input shape
inputs = keras.Input(shape=(input_size,1), batch_size=1)

model = NeuralNetwork(input_size, conv1d_filters, conv1d_strides, hidden_units, learning_rate)


model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
model.summary()

## Train the model

In [ ]:
history = model.fit(x=X_random_training, y=y_random_training, epochs=epochs, batch_size=batch_size, validation_data=(X_ordered_testing, y_ordered_testing)) 

In [ ]:
model.save_weights('models/'+name+'/'+name)

## Run predictions
### 0. Load the model

In [ ]:
model.load_weights('models/'+name+'/'+name)

### 1. On the test audio data

In [ ]:
# Run Prediction #################################################
# Test the model on the testing data #############################

print("Running prediction..")
prediction = model.predict(X_ordered_testing)

save_wav('models/'+name+'/y_pred.wav', prediction)
save_wav('models/'+name+'/x_test.wav', X_testing)
save_wav('models/'+name+'/y_test.wav', y_testing)

print("X_testing shape: ", X_testing.shape)
print("X_ordered_testing shape: ", X_ordered_testing.shape)
print("y_testing shape: ", y_testing.shape)
print("prediction shape: ", prediction.shape)

print("Note that the prediction shape is smaller than the y_testing shape. This is because the first predicted sample needs input_size samples for prediction.\n")

### 2. On a number sequence (to control inference)

In [ ]:
# Test the model simple number sequence to compare with inference #
X_testing_2 = np.array([])

for i in range(0, 300):
    X_testing_2 = np.append(X_testing_2, i*0.001)

X_testing_2 = np.expand_dims(X_testing_2, axis=0)
X_testing_2 = np.expand_dims(X_testing_2, axis=0)

X_testing_2 = np.reshape(X_testing_2, (2, 150, 1))

print("Running prediction..")
prediction_2 = model.predict(X_testing_2)
print(f"prediction {prediction_2}")

print("X_testing_2 shape: ", X_testing_2.shape)
print("prediction_2 shape: ", prediction_2.shape)

## Export as tflite model

In [ ]:
# tflite does not support dynamic batch sizes with default operations, so we would need to add the flex ops library

buffer_sizes = [64, 128, 256, 512, 1024, 2048, 4096, 8192]
model_input_size = 150
# rf is the receptive field and hence the kernel size of the last layer because of the dilation
# hence the model input size needs to be at least rf
for buffer_size in buffer_sizes:
    batch_size = buffer_size
    input_shape = [batch_size, model_input_size, 1]

    func = tf.function(model).get_concrete_function(
        tf.TensorSpec(input_shape, dtype=tf.float32))
    converter = tf.lite.TFLiteConverter.from_concrete_functions([func])
    tflite_model = converter.convert()

    # Save the model.
    with open("models/"+name+"/"+"GuitarLSTM-"+str(buffer_size)+".tflite", 'wb') as f:
      f.write(tflite_model)

## Test as tflite model

In [ ]:
interpreter = tf.lite.Interpreter(model_path="models/"+name+"/GuitarLSTM-256.tflite")
interpreter.allocate_tensors()  # Needed before execution!
output = interpreter.get_output_details()[0]  # Model has single output.
input = interpreter.get_input_details()[0]  # Model has single input.
print(f"Input shape: {input['shape']}")
print(f"Output shape: {output['shape']}")

In [ ]:
# Test the model simple number sequence to compare with inference #
X_testing_3 = np.array([])

for i in range(0, 256*150):
    X_testing_3 = np.append(X_testing_3, i*float(0.000001))

X_testing_3 = np.expand_dims(X_testing_3, axis=0)
X_testing_3 = np.expand_dims(X_testing_3, axis=0)

X_testing_3 = np.reshape(X_testing_3, (256, 150, 1))

X_testing_3 = X_testing_3.astype(np.float32)

interpreter.set_tensor(input['index'], X_testing_3)
interpreter.invoke()
prediction_3 = interpreter.get_tensor(output['index'])

print("Running prediction..")
print(f"prediction {prediction_3}")

print("X_testing_3 shape: ", X_testing_3.shape)
print("prediction_3 shape: ", prediction_3.shape)

In [ ]:
# Run Prediction #################################################
# Test the model on the testing data #############################

print("Running prediction..")
print("X_ordered_testing shape: ", X_ordered_testing.shape)
num_batches = X_ordered_testing.shape[0] // 256
prediction_4 = np.zeros((num_batches*256,1))
for i in range(num_batches):
    interpreter.set_tensor(input['index'], X_ordered_testing[i*256:(i+1)*256])
    interpreter.invoke()
    prediction_4[i*256:(i+1)*256] = interpreter.get_tensor(output['index'])
    print(f"Batch {i+1}/{num_batches}")

save_wav('models/'+name+'/y_pred_tflite.wav', prediction_4)

print("X_testing shape: ", X_testing.shape)
print("X_ordered_testing shape: ", X_ordered_testing.shape)
print("y_testing shape: ", y_testing.shape)
print("prediction_4 shape: ", prediction_4.shape)

print("Note that the prediction shape is smaller than the y_testing shape. This is because the first predicted sample needs input_size samples for prediction.\n")

###

In [ ]:
for buffer_size in buffer_sizes:
    batch_size = buffer_size
    input_shape = [batch_size, model_input_size, 1]

    # Define the input shape
    input_signature = [tf.TensorSpec(input_shape, tf.float32, name='x')]

    # Convert the model
    onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=18)
    onnx.save(proto=onnx_model, f="models/"+name+"/"+"GuitarLSTM-tflite-"+str(buffer_size)+".onnx")